# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [19]:
# imports
import requests
import json
import ollama
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

In [2]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'
MODEL_DEEPSEEK = 'deepseek-r1:32b'

In [3]:
# set up environment
OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}

In [46]:
# here is the question; type over this to ask something new
def language_selection(language="English"):
    """
    Returns the system prompt in the selected language
    Args:
        language (str): Language for the response (default: "English")
    """
    prompts = {
        "Spanish": "Eres un asistente útil que responde preguntas técnicas en español. Termina con una analogía del código proporcionado.",
        "English": "You are a helpful assistant that answers technical questions in English. End with an analogy of the provided code.",
        "French": "Vous êtes un assistant utile qui répond aux questions techniques en français. Terminez avec une analogie du code fourni.",
        "Portuguese": "Você é um assistente útil que responde perguntas técnicas em português. Termine com uma analogia do código fornecido.",
        # Add more languages as needed
    }
    return prompts.get(language, prompts["English"])  # Default to English if language not found

def get_explanation(question, language="English"):
    """
    Get explanation in the selected language
    Args:
        question (str): The technical question to answer
        language (str): Desired language for the response
    """
    messages = [
        {"role": "system", "content": language_selection(language)},
        {"role": "user", "content": question}
    ]
    
    response = ollama.chat(model=MODEL_DEEPSEEK, messages=messages)
    return response['message']['content']

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

payload = {
  "model": MODEL_LLAMA,
  "messages": messages,
  "stream": False
}



In [ ]:
# Get gpt-4o-mini to answer, with streaming

In [49]:
# Get Llama 3.2 to answer
#response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)
#content = response.json()['message']['content']
#print(response.json()['message']['content'])
#display(Markdown(content))

# Use ollama to answer
# Get response in different languages
display(Markdown(get_explanation(question, "English")))
#print("Respuesta en Español")
display(Markdown(get_explanation(question, "Spanish")))
display(Markdown(get_explanation(question, "Portuguese")))

**Code Explanation**

The given code snippet is a part of a Python function that uses a combination of `yield from` and generator expressions to extract author names from a list of books.

Here's a breakdown:

- `{... for ...}`: This is a generator expression, which allows us to iterate over the elements in the list without loading the entire list into memory.
  
- `"book.get("author")"`: For each book in the list `books`, this gets the author's name from the dictionary using the `.get()` method, which returns `None` if the key does not exist.

- `yield from {...}`: This is used to delegate the iteration over the inner generator expression to another iterable (in this case, the result of the generator expression).

So, when we put it all together:

```python
for author in yield from {book.get("author") for book in books if book.get("author")}:
    # Do something with the authors...
```

**Why**

The code is likely being used in a scenario where we have a list of books, and each book is represented as a dictionary containing information about the book. The author's name might not always be present in these dictionaries (in which case `.get("author")` would return `None`).

By using this generator expression with `yield from`, we're creating an iterator that yields only the authors whose names exist in the books' dictionaries, without having to load all the book data into memory at once.

**Analogy**

This code is like a librarian searching for books by author. Instead of looking up every single book's author one by one (which would be inefficient), our generator expression looks over the list of books quickly, and then yields only the authors it finds. The `yield from` part is like saying to the librarian: "Please find me all the authors you can in those books, but don't bring them to me just yet - I'll take care of that here." Then, we're able to iterate over just those authors as needed, one by one.

¡Hola! Estoy aquí para ayudarte a entender este código.

Este código utiliza un concepto llamado "coroutines" en Python, que se enfoca en la gestión de las interrupciones y la suspensión del ejecución de una función.

Vamos a analizarlo paso a paso:

1. `yield from`: esta palabra clave permite que el código llame a otra función (en este caso, un generador) y continúe su ejecución desde donde lo dejó.
2. `{book.get("author") for book in books if book.get("author")}`: Esta es una expresión genérica que utiliza un patrón de iteración llamado "generador" o "coroutine". Es similar a un bucle `for`, pero en lugar de iterar sobre una lista o tupla, se ejecuta una función y devuelve una secuencia de resultados.

En este caso, la función genérica (el generador) itera sobre el conjunto `books`. Para cada libro (`book`), llama a la función `get("author")` para obtener la autoría del libro. Sin embargo, solo incluye en la secuencia los libros que tienen una autoría válida (es decir, no es None).

El resultado es un generador que itera sobre una secuencia de autorías.

Ahora, volvemos al punto `yield from`. Esta palabra clave permite que el código llame a este generador y lo ejecute como si fuera una función normal. En otras palabras, cuando se llama a esta función, su ejecución se suspende en cada iteración del generador y se vuelve a continuar desde donde lo dejó.

Por ejemplo, si `books` es un conjunto con varios libros, el resultado sería un generador que itera sobre las autorías de los libros. Si luego llamamos a esta función con un `yield from`, la secuencia de autorías se devolverá, permitiendo que cada autoría sea ejecutada por separado.

**Analogía del código:**

Imagina que tienes una biblioteca con varios libros y deseas obtener las autorías de todos ellos. En lugar de leer cada libro uno a uno, puedes utilizar un generador para iterar sobre los libros y obtener sus autorías de manera concurrente.

De la misma manera, el código `yield from` permite que el código llame a este generador como si fuera una función normal, lo que te permite ejecutar las autorías individualmente.

Esse código é um exemplo de usar a sintaxe "yield from" no Python, que permite criar iteráveis compostos.

Aqui está uma explicação detalhada:

- `yield from`: Essa palavra-chave é usada para criar uma nova sequência que contém todos os elementos de uma ou mais sequências antigas.
- `{book.get("author") for book in books if book.get("author")}`: Isso é um generator expression (expressão geradora), que cria um objeto iterator que gera valores em tempo real. O generator expression é semelhante a uma expressão de lista, mas em vez de criar todos os elementos e armazená-los em memória, ele os gera quando necessário.

O código inteiro está fazendo o seguinte:

- Seleciona apenas os livros que têm um autor (ou seja, não é None)
- Para cada livro que atende a esse critério, gera o autor do livro

Portanto, `yield from {book.get("author") for book in books if book.get("author")}` é uma forma de criar um iterator que retorna os autores dos livros que têm um autor.

Uma analogia para explicar melhor: se você estivesse sortindo um monte de cartas com as informações do livro, como o título e o autor, essa linha de código seria como sortear apenas as cartas com autor (ou seja, não sortando aquelas sem autor). Em vez de armazenar todas as cartas sorteadas em uma lista, ele as gera na hora de ser necessário, como se estivesse lendo cada carta do monte um por um.